In [ ]:
import numpy as np
import tensorflow as tf

# Graph.
vif = tf.placeholder(tf.float32, shape=[2])

#out = vif + tf.cast(tf.cast(vif, tf.int32), tf.float32)
out = vif + tf.floor(vif)
grad = tf.gradients(tf.reduce_sum(out), vif)

ceil = tf.ceil(vif)
floor = tf.floor(vif)
trunk = tf.minimum(tf.maximum(vif, -1), 1)

# Session.
f = np.array([2.99, -1.1], dtype='float32')
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
#print (sess.run(grad, feed_dict={vif: f}))  # print 2
print (sess.run([floor, ceil, trunk], feed_dict={vif: f}))  # print 2


In [ ]:
# Different approach - change memory "orientation"

import tensorflow as tf

# Reset graph - just in case.
tf.reset_default_graph()


# place holders 
a = tf.placeholder(tf.float32, shape=None)
b = tf.placeholder(tf.float32, shape=None)

op = a * b
op2 = tf.tensordot(tf.transpose(a), b, axes=1)

# Finally - initialize all variables.
initialize_model = tf.global_variables_initializer()    
    
# Values
va= [[1,2,3],[2,3,4]]
vb= [[-1,-2,-3],[2,3,4]]

# Initialize session.
sess=tf.InteractiveSession()
sess.run(initialize_model)
# Execute graph.
print("a*b =\n",sess.run([op], feed_dict={a:va, b:vb}))
print("a dot b =\n",sess.run([op2], feed_dict={a:va, b:vb}))


In [ ]:
# Sorts 2D tensor, each row separatelly.
a = tf.Variable([[0.51, 0.52, 0.53, 0.94, 0.35],
             [0.32, 0.72, 0.83, 0.74, 0.55],
             [0.23, 0.72, 0.63, 0.64, 0.35],
             [0.11, 0.02, 0.03, 0.14, 0.15],
             [0.01, 0.72, 0.73, 0.04, 0.75]],tf.float32)

row_size = a.get_shape().as_list()[-1]
top_k = tf.nn.top_k(-a, k=row_size)

# Finally - initialize all variables.
initialize_model = tf.global_variables_initializer()    
    
# Execute graph.
sess=tf.InteractiveSession()
# Initialize.
sess.run(initialize_model)

print(sess.run(-top_k.values))
sess.close()

In [ ]:
# How to perform operation on arrays.
elems = np.array([1, 2, 3, 4, 5, 6])
#squares = tf.map_fn(lambda x: x * x, elems)
squares = tf.multiply(elems, elems)
# squares == [1, 4, 9, 16, 25, 36]

sess=tf.Session()
print(sess.run(squares))
sess.close()

In [ ]:
elems = (np.array([1, 2, 3]), np.array([-1, 1, -1]))
alternate = tf.map_fn(lambda x: x[0] * x[1], elems, dtype=tf.int64)
# alternate == [-1, 2, -3]

sess=tf.Session()
print(sess.run(alternate))
sess.close()

In [ ]:
# "Broadcasting ability of TF"
import tensorflow as tf

# Reset graph - just in case.
tf.reset_default_graph()

x = tf.constant([[0, 1,2],[2, 3,2],[4, 5,2],[6, 7,2]], dtype=tf.float32)
y = tf.constant([[0, 1,2],[-2, -1,2]], dtype=tf.float32)

x_vect_len = int(x.shape[1])
print("len=",x_vect_len)
x_ = tf.expand_dims(x, 0)
y_ = tf.expand_dims(y, 1)
z = tf.reshape(tf.add(x_, y_), [-1, x_vect_len])

sess = tf.InteractiveSession()
sess.run(z)
print(x_.eval())
print(y_.eval())

sess.close()

In [ ]:
# Reset graph - just in case.
tf.reset_default_graph()

# place holders 
batch = tf.placeholder(tf.float32, shape=[3,3], name="batch")
memory = tf.placeholder(tf.float32, shape=[2,3], name="memory")
# Cos similarity
norm_batch = tf.nn.l2_normalize(batch,0) 
norm_memory = tf.nn.l2_normalize(memory,0)

nb_ = tf.expand_dims(norm_batch, 0)
nm_ = tf.expand_dims(norm_memory, 1)
cos_similarity = tf.reshape(tf.add(nb_,nm_), [-1, 2])
#cos_similarity = tf.reshape(tf.reduce_sum(tf.multiply(nb_,nm_)), [-1, 2])

                            
sess=tf.InteractiveSession()
cos_sim=sess.run(cos_similarity,feed_dict={batch:[[1,2,3],[3,3,3],[1,2,6]],memory:[[2,4,6],[1,2,4]]})

print(cos_sim)

sess.close()

In [ ]:
# Reset graph - just in case.
tf.reset_default_graph()

# Size of the hidden state 64
HIDDEN_SIZE = 3

# Size of the MANN memory.
MEMORY_SIZE = 5

# A batch size of 100
BATCH_SIZE = 2

# A single recurrent layer of number of units = sequences of length
# e.g. 200 bytes
SEQ_LENGTH = 10

EPS = 1e-15

# place holders 
batch = tf.placeholder(tf.float32, shape=None, name="batch")
#memory = tf.placeholder(tf.float32, shape=[None], name="memory")
memory = tf.Variable(tf.zeros([MEMORY_SIZE, HIDDEN_SIZE]), trainable=False, name="memory")

memory_set = memory.assign([[1, 0, 1],[0.1, 0.2 , 0.4],[ 0,0,0],[-0.3,0.2,0.3],[0, 1, 0]])

# Create BATCH_SIZE placeholders for similarity - each MEMORY_SIZE x 1,  
with tf.name_scope("similarity"):
  # Define similarity buffers.
  similarity = list()
  #for b in range(BATCH_SIZE):
    # Collect placeholders for similarity.
    #similarity.append(tf.placeholder(tf.float32, shape=[MEMORY_SIZE, 1], name="Similarity"))
 
  # Normalize
  norm_batch = tf.nn.l2_normalize(batch,1) 
  norm_memory = tf.nn.l2_normalize(memory,1)
  print(norm_batch[0])
  # Define similarity buffers.
  numerator_batch = list()
  denominator_batch = list()
  similarity_batch = list()

  for b in range(BATCH_SIZE):
    similarity_batch.append(tf.map_fn(lambda x: tf.reduce_sum(tf.multiply(norm_batch[b],x)), norm_memory))
    # Read weight based on similarity.
  read_weight = tf.nn.softmax(similarity_batch)
    
sess=tf.InteractiveSession()
# Initialize.
sess.run(memory_set)
print("memory =",memory.eval())

tmp_batch = [[1, 0, 1],[-0.3,0.3,0.3]]

print("Batch=",batch.eval(feed_dict={batch:tmp_batch}))
print("Norm batch=",norm_batch.eval(feed_dict={batch:tmp_batch}))


num, den, sim, rw=sess.run([numerator_batch, denominator_batch, similarity_batch, read_weight],
                           feed_dict={batch:tmp_batch})
print("norm memory =",norm_memory.eval())

print("numerator_batch=",num)
print("denominator_batch=\n",den,"\n")

print("similarity=",sim)
print("rw=", rw)


sess.close()


In [8]:
# RGB coloring of matrices (hot-cold) for visualization purposes.
import tensorflow as tf

HIDDEN_SIZE = 9
MEMORY_SLOTS = 2
EPS = 1e-10

# Reset graph - just in case.
tf.reset_default_graph()

memory = tf.transpose([[1.0, 0, 1, 2, 3, 4, 5, 6, 1],[-0.2, 0.1, -0.4, 0.5, 0.1, 0.9, 0.8, 0.3, -0.3]])

zeros = tf.zeros_like(memory)
# Get negative values only.
neg = tf.less(memory, zeros)
blue = tf.multiply(tf.cast(neg, tf.float32), memory)
min_blue = tf.reduce_min(memory, axis=None, keep_dims=True) + EPS
norm_blue = 255.0 * blue/min_blue
# Get positive values only.
pos = tf.greater(memory, zeros)
red = tf.multiply(tf.cast(pos, tf.float32), memory)
max_red = tf.reduce_max(memory, axis=None, keep_dims=True) + EPS
norm_red = 255.0 * red/max_red
# Stack them into three channel image with hot-cold values.
rgb_memory = tf.stack([norm_red, zeros, norm_blue], axis=2)
rgb_memory_reshaped = tf.reshape(rgb_memory, [1, HIDDEN_SIZE, MEMORY_SLOTS, 3])



# Finally - initialize all variables.
initialize_model = tf.global_variables_initializer()    
    
# Execute graph.
sess=tf.InteractiveSession()
# Initialize.
sess.run(initialize_model)
print("memory =\n", sess.run([memory]))
print("min_blue =\n", sess.run([min_blue]))
print("max_red =\n", sess.run([max_red]))

print("rgb_memory =\n",sess.run([rgb_memory]))
print("rgb_memory_reshaped =\n",sess.run([rgb_memory_reshaped]))


memory =
 [array([[ 1.        , -0.2       ],
       [ 0.        ,  0.1       ],
       [ 1.        , -0.40000001],
       [ 2.        ,  0.5       ],
       [ 3.        ,  0.1       ],
       [ 4.        ,  0.89999998],
       [ 5.        ,  0.80000001],
       [ 6.        ,  0.30000001],
       [ 1.        , -0.30000001]], dtype=float32)]
min_blue =
 [array([[-0.40000001]], dtype=float32)]
max_red =
 [array([[ 6.]], dtype=float32)]
rgb_memory =
 [array([[[  42.5 ,    0.  ,   -0.  ],
        [  -0.  ,    0.  ,  127.5 ]],

       [[   0.  ,    0.  ,   -0.  ],
        [   4.25,    0.  ,   -0.  ]],

       [[  42.5 ,    0.  ,   -0.  ],
        [  -0.  ,    0.  ,  255.  ]],

       [[  85.  ,    0.  ,   -0.  ],
        [  21.25,    0.  ,   -0.  ]],

       [[ 127.5 ,    0.  ,   -0.  ],
        [   4.25,    0.  ,   -0.  ]],

       [[ 170.  ,    0.  ,   -0.  ],
        [  38.25,    0.  ,   -0.  ]],

       [[ 212.5 ,    0.  ,   -0.  ],
        [  34.  ,    0.  ,   -0.  ]],

       [[ 255. 

In [ ]:
import tensorflow as tf

# Reset graph - just in case.
tf.reset_default_graph()

def rest(x): return tf.gather(x, tf.range(1, tf.size(x)))

x = tf.Variable([0., 1.])
x_op = tf.assign(x, [0., 1., 2.], validate_shape=False)

with tf.control_dependencies([x_op]):
    true_fun  = lambda: tf.assign(x, rest(x_op), validate_shape=False)
    false_fun = lambda: tf.constant([])
    pred = tf.constant(True)
    cond_op = tf.cond(pred, true_fun, false_fun)

with tf.Session(""):
  x.initializer.run()
  print(cond_op.eval())

In [ ]:
# Finds given top elements
import tensorflow as tf
# Reset graph - just in case.
tf.reset_default_graph()

tmp_batch = [[1, 0, 1, 2, 3, 4, 5, 6, 1],[-0.2, 0.1, -0.4, 0.5, 0.1, 0.9, 0.8, 0.3, -0.3]]


top_k = tf.nn.top_k(tmp_batch, 3)

# Finally - initialize all variables.
initialize_model = tf.global_variables_initializer()    
    
# Execute graph.
sess=tf.InteractiveSession()
# Initialize.
sess.run(initialize_model)
print(sess.run([top_k]))



## Finds n smallest elements - using additional division into list
### (~ok, does not handle several elements with the same values)

In [ ]:
# Finds n smallest elements

import tensorflow as tf
# Reset graph - just in case.
tf.reset_default_graph()

batch = tf.convert_to_tensor([[4, 3, 2, 1], [10, 20, 1, 40]])
print("batch.shape=", batch.shape)

# Number of smallest elements.
n = 3

# You can do it using built-in tf.nn.top_k function - find size-n top elements in each sample.
k_number = batch.shape[1] - n
print("k_number=", k_number)

smallest_ones_batch = list()
for i_batch in range(batch.shape[0]):
    top = tf.nn.top_k(batch[i_batch], k_number)
    # To get boolean True/False values, you can first get the k-th value and then use tf.greater_equal:
    kth = tf.reduce_min(top.values)
    top2 = tf.greater_equal(batch[i_batch], kth)

    # And finally - cast it to n smallest elements.
    smallest = tf.cast(top2, tf.float32) * -1.0 + 1.0
    
    smallest_ones_batch.append(smallest)


#w_prew = tf.convert_to_tensor()

# Execute graph.e
sess=tf.InteractiveSession()
print("batch=",sess.run(batch))

print("smallest_ones=",sess.run(smallest_ones_batch))


## Finds n smallest elements - without the division (OK)

In [ ]:
# Finds n smallest elements

import tensorflow as tf
# Reset graph - just in case.
tf.reset_default_graph()

batch = tf.convert_to_tensor([[4, 3, 2, 1], [10, 20, 1, 40]])
#batch = tf.convert_to_tensor([[ 0.1143328,   0.31078878,  0.1143328,   0.1143328,   0.23188008,  0.1143328 ],
#    [ 0.1143328,   0.1143328,   0.31078878,  0.1143328,   0.1143328,   0.23188008],
#    [ 0.1143328,   0.31078878,  0.1143328,   0.1143328,   0.23188008,  0.1143328 ]])
#print("batch.shape=", batch.shape)

# Number of smallest elements.
n = 3

# You can do it using built-in tf.nn.top_k function - find size-n top elements ALONG THE LAST DIMENSION
top = tf.nn.top_k(-batch, n)
# To get boolean True/False values, you can first get the k-th value and then use tf.greater_equal:
kth = tf.reduce_min(top.values, axis=1, keep_dims=True)
# And finally - cast it to n smallest elements.
smallest_ones_batch = tf.cast(tf.greater_equal(-batch, kth), tf.float32)

# Execute graph.e
sess=tf.InteractiveSession()
print("batch=",sess.run(batch))

#print("top.values=",sess.run(top.values))

#print("kth=",sess.run(kth))

print("smallest_ones=",sess.run(smallest_ones_batch))


## Different approach - change memory "orientation" (BAD!)
### Fully operational read-write-update heads, batch of vectors

In [ ]:
import tensorflow as tf

# Reset graph - just in case.
tf.reset_default_graph()

# Size of the hidden state 64
HIDDEN_SIZE = 3

# Size of the MANN memory.
MEMORY_SIZE = 5

# A batch size of 100
#BATCH_SIZE = 2

# A single recurrent layer of number of units = sequences of length
# e.g. 200 bytes
SEQ_LENGTH = 10

# "Read decay".
GAMMA = 0.1

# Number of smallest elements.
N_SMALLEST =2

#EPS = 1e-15

# place holders 
batch = tf.placeholder(tf.float32, shape=None, name="Batch_h")
#memory = tf.placeholder(tf.float32, shape=[None], name="memory")
memory = tf.Variable(tf.zeros([HIDDEN_SIZE, MEMORY_SIZE]), trainable=False, name="Memory_M")
#alpha = tf.Variable(tf.truncated_normal(shape=[1]), name="Alpha")
alpha = tf.Variable(tf.truncated_normal(shape=[1]), name="Alpha")

# SET INITIAL MEMORY STATE.
memory_set = memory.assign(tf.transpose([[1, 0, 1],
                            [0.1, 0.2, 0.4],
                            [ 0, 0.3, 0],
                            [-0.3, 0.2, 0.3],
                            [0, 1, 0]]))
alpha_set = alpha.assign([0.1])

# Placeholders for previous weights.
prev_update_weights = tf.placeholder(tf.float32, shape=None, name="Prev_uw")
prev_read_weights = tf.placeholder(tf.float32, shape=None, name="Prev_rw")


with tf.name_scope("Read_head"):
    # Normalize batches and memory.
    norm_batch = tf.nn.l2_normalize(batch,1, name="NormalizedBatch_h") 
    norm_memory = tf.nn.l2_normalize(memory,1, name="NormalizedMemory_h")

    # calculate similarity.
    similarity = tf.tensordot(norm_batch, norm_memory, axes=1, name= "Similarity_D") 
    # Read weights based on similarity.
    read_weights = tf.nn.softmax(similarity, name="Read_weights_rw")
    # Read "vector" (in fact batch).
    r = tf.tensordot(read_weights, tf.transpose(memory), axes=1, name="Read_vector_r")

# TODO: add dependencies, that write will be done after read.
with tf.name_scope("Write_head"):
    # A "truncation scheme to update the least-used positions".
    # First, find (size-n) top elements (in each "batch sample"/head separatelly).
    k_number = MEMORY_SIZE - N_SMALLEST
    print(prev_update_weights.shape)
    top = tf.nn.top_k(prev_update_weights, k_number)

    # To get boolean True/False values, you can first get the k-th value and then use tf.greater_equal:
    kth = tf.reduce_min(top.values)
    top2 = tf.greater(prev_update_weights, kth)
    # And finally - cast it to n smallest elements.
    smallest_lru_weights = tf.cast(top2, tf.float32) * -1.0 + 1.0

    write_weights = tf.add(tf.sigmoid(alpha) * prev_read_weights, (1.0 - tf.sigmoid(alpha)) * smallest_lru_weights, 
                           name="Write_weights_ww")
    
with tf.name_scope("Memory_update"):
    calculated_mem_update = tf.tensordot(tf.transpose(batch), write_weights, axes=1)
    memory_update_op = memory.assign(memory + calculated_mem_update)

with tf.name_scope("Update_head"): # This relies on prev. weights and will be used in fact in NEXT step.
    update_weights = tf.add(GAMMA * prev_update_weights, read_weights + write_weights, name="Update_weights_uw")

    
    
# Finally - initialize all variables.
initialize_model = tf.global_variables_initializer()    
    
# Execute graph.
sess=tf.InteractiveSession()
# Initialize.
sess.run(initialize_model)
sess.run([memory_set, alpha_set])
print("Memory =\n",memory.eval())

#tmp_batch = [[1, 0, 1],[-0.3, 0.3, -0.3]]
tmp_batch = [[0.1, 0, 0],[0, -0.1, 0]]
# Prev UW [batch size x memory size]
prev_uw = [[0,0,0,0,0], [0,0,0,0,0]]
prev_rw = [[0,0,0,0,0], [0,0,0,0,0]]

print("\nBatch=\n",batch.eval(feed_dict={batch:tmp_batch}))
#print("Norm batch=",norm_batch.eval(feed_dict={batch:tmp_batch}))

for i in range(10):
    sim, r_vect, prev_rw, prev_uw, prev_ww, mu_op = sess.run(
        [similarity, r, read_weights, update_weights, write_weights, memory_update_op],
                               feed_dict={
                                   batch:tmp_batch,
                                   prev_update_weights: prev_uw,
                                   prev_read_weights: prev_rw
                               })
    #print("Norm memory =\n",norm_memory.eval())
    print ("\n=====i=", i)
    print("Memory after update=\n", mu_op)
    #print("\nSimilarity=\n",sim)
    print("\nprev_rw=\n", prev_rw)
    #print("\nprev_uw=\n", prev_uw)
    print("\nprev_ww=\n", prev_ww)

    print("\n readed vector=\n", r_vect)


sess.close()

## Different approach - change memory "orientation"
### Read-write-update heads, step towards batch of "sequences of vectors".

In [ ]:
import tensorflow as tf

# Reset graph - just in case.
tf.reset_default_graph()

# Size of the hidden state 64
HIDDEN_SIZE = 3

# Size of the MANN memory.
MEMORY_SIZE = 5

# A batch size of 100
#BATCH_SIZE = 2

# A single recurrent layer of number of units = sequences of length
# e.g. 200 bytes
SEQ_LENGTH = 10

# "Read decay".
GAMMA = 0.1

# Number of smallest elements.
N_SMALLEST =2

#EPS = 1e-15

# place holders 
batch = tf.placeholder(tf.float32, shape=None, name="Batch_h")
#memory = tf.placeholder(tf.float32, shape=[None], name="memory")
memory = tf.Variable(tf.zeros([HIDDEN_SIZE, MEMORY_SIZE]), trainable=False, name="Memory_M")
#alpha = tf.Variable(tf.truncated_normal(shape=[1]), name="Alpha")
alpha = tf.Variable(tf.truncated_normal(shape=[1]), name="Alpha")

# SET INITIAL MEMORY STATE.
memory_set = memory.assign(tf.transpose([[1, 0, 1],
                            [0.1, 0.2, 0.4],
                            [ 0, 0.3, 0],
                            [-0.3, 0.2, 0.3],
                            [0, 1, 0]]))
alpha_set = alpha.assign([0.1])

# Placeholders for previous weights.
prev_update_weights = tf.placeholder(tf.float32, shape=None, name="Prev_uw")
prev_read_weights = tf.placeholder(tf.float32, shape=None, name="Prev_rw")


with tf.name_scope("Read_head"):
    # Normalize batches and memory.
    norm_batch = tf.nn.l2_normalize(batch,1, name="NormalizedBatch_h") 
    norm_memory = tf.nn.l2_normalize(memory,1, name="NormalizedMemory_h")

    # calculate similarity.
    similarity = tf.tensordot(norm_batch, norm_memory, axes=1, name= "Similarity_D") 
    # Read weights based on similarity.
    read_weights = tf.nn.softmax(similarity, name="Read_weights_rw")
    # Read "vector" (in fact batch).
    r = tf.tensordot(read_weights, tf.transpose(memory), axes=1, name="Read_vector_r")

# TODO: add dependencies, that write will be done after read.
with tf.name_scope("Write_head"):
    # A "truncation scheme to update the least-used positions".
    # First, find (size-n) top elements (in each "batch sample"/head separatelly).
    k_number = MEMORY_SIZE - N_SMALLEST
    print(prev_update_weights.shape)
    top = tf.nn.top_k(prev_update_weights, k_number)

    # To get boolean True/False values, you can first get the k-th value and then use tf.greater_equal:
    kth = tf.reduce_min(top.values)
    top2 = tf.greater_equal(prev_update_weights, kth)
    # And finally - cast it to n smallest elements.
    smallest_lru_weights = tf.cast(top2, tf.float32) * -1.0 + 1.0

    write_weights = tf.add(tf.sigmoid(alpha) * prev_read_weights, (1.0 - tf.sigmoid(alpha)) * smallest_lru_weights, 
                           name="Write_weights_ww")
    
with tf.name_scope("Memory_update"):
    calculated_mem_update = tf.tensordot(tf.transpose(batch), write_weights, axes=1)
    memory_update_op = memory.assign(memory + calculated_mem_update)

with tf.name_scope("Update_head"): # This relies on prev. weights and will be used in fact in NEXT step.
    update_weights = tf.add(GAMMA * prev_update_weights, read_weights + write_weights, name="Update_weights_uw")

    
    
# Finally - initialize all variables.
initialize_model = tf.global_variables_initializer()    
    
# Execute graph.
sess=tf.InteractiveSession()
# Initialize.
sess.run(initialize_model)
sess.run([memory_set, alpha_set])
print("Memory =\n",memory.eval())

#tmp_batch = [[1, 0, 1],[-0.3, 0.3, -0.3]]
tmp_batch = [[0.1, 0, 0],[0, 0, 0]]
# Prev UW [batch size x memory size]
prev_uw = [[0,0,0,0,0], [0,0,0,0,0]]
prev_rw = [[0,0,0,0,0], [0,0,0,0,0]]

print("\nBatch=\n",batch.eval(feed_dict={batch:tmp_batch}))
#print("Norm batch=",norm_batch.eval(feed_dict={batch:tmp_batch}))

for i in range(10):
    sim, r_vect, prev_rw, prev_uw, prev_ww, mu_op = sess.run(
        [similarity, r, read_weights, update_weights, write_weights, memory_update_op],
                               feed_dict={
                                   batch:tmp_batch,
                                   prev_update_weights: prev_uw,
                                   prev_read_weights: prev_rw
                               })
    #print("Norm memory =\n",norm_memory.eval())
    print ("\n=====i=", i)
    print("Memory after update=\n", mu_op)
    #print("\nSimilarity=\n",sim)
    print("\nprev_rw=\n", prev_rw)
    #print("\nprev_uw=\n", prev_uw)
    print("\nprev_ww=\n", prev_ww)

    print("\n readed vector=\n", r_vect)


sess.close()